In [2]:
import os
from os import listdir
from os.path import isfile, join
import csv

import pandas as pd
import tensorflow as tf

# from google.colab import drive

In [32]:
# upload retrained_graph.pb, retrained_labels.txt
# upload folder data with images

In [49]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# folder containing the images
image_folder = 'data'
image_csv = 'images.csv'

In [4]:
# collect file paths and names
image_data = []
for root, _, files in os.walk(image_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
            file_path = os.path.join(root, file)
            image_data.append([file_path, file])

In [5]:
# Debugging: Check if any images were found
if not image_data:
    print(f"No image files found in the folder: {image_folder}")
else:
    print(f"Found {len(image_data)} image files.")

Found 7219 image files.


In [37]:
try:
    # Write to CSV
    with open(image_csv, mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(['image_path', 'image_name'])  # Header
        writer.writerows(image_data)
    print(f"CSV file created at: {image_csv}")
except Exception as e:
    print(f"Error writing to CSV: {e}")

CSV file created at: images.csv


In [38]:
# Load image paths from CSV
df = pd.read_csv("images.csv")

In [39]:
df.head()

,image_path,image_name
0,data/1 (31).jpeg,1 (31).jpeg
1,data/1 (25).jpeg,1 (25).jpeg
2,data/1 (8).jpeg,1 (8).jpeg
3,data/1 (6).jpeg,1 (6).jpeg
4,data/1 (9).jpeg,1 (9).jpeg


In [40]:
# Make sure the csv has a column 'image_path'
image_paths = df["image_path"].tolist()

In [41]:
# Load label names (male, female)
label_lines = [line.rstrip() for line in tf.io.gfile.GFile("retrained_labels.txt")]

In [42]:
# Load trained model
with tf.io.gfile.GFile("retrained_graph.pb", 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')

In [43]:
# Make predictions
predictions_list = []

In [44]:
with tf.compat.v1.Session() as sess:
    softmax_tensor = sess.graph.get_tensor_by_name('final_result:0')

    for image_path in image_paths:
        image_data = tf.io.gfile.GFile(image_path, 'rb').read()
        predictions = sess.run(softmax_tensor, {'DecodeJpeg/contents:0': image_data})

        # Get the most confident prediction
        top_label_index = predictions[0].argsort()[-1]
        predicted_label = label_lines[top_label_index]

        # Store results
        predictions_list.append({"image_path": image_path, "predicted_label": predicted_label})

In [45]:
predictions_list[:5]

[{'image_path': 'data/1 (31).jpeg', 'predicted_label': 'male'},
 {'image_path': 'data/1 (25).jpeg', 'predicted_label': 'female'},
 {'image_path': 'data/1 (8).jpeg', 'predicted_label': 'male'},
 {'image_path': 'data/1 (6).jpeg', 'predicted_label': 'male'},
 {'image_path': 'data/1 (9).jpeg', 'predicted_label': 'male'}]

In [51]:
# Save predictions to CSV
results_df = pd.DataFrame(predictions_list)
results_df.to_csv("classified_images.csv", index=False)

In [16]:
results_df.head()

,image_path,predicted_label
0,data/1 (31).jpeg,male
1,data/1 (25).jpeg,female
2,data/1 (8).jpeg,male
3,data/1 (6).jpeg,male
4,data/1 (9).jpeg,male


In [17]:
print("Classification complete! Results saved in 'classified_images.csv'.")

Classification complete! Results saved in 'classified_images.csv'.


In [ ]:
results_df.to_csv("/content/drive/My Drive/classified_images.csv", index=False)

In [ ]:
df_verify = pd.read_csv("/content/drive/My Drive/classified_images.csv")

In [ ]:
df_verify.head()

,image_path,predicted_label
0,data/1 (1565).jpg,male
1,data/1 (2324).jpg,male
2,data/1 (1953).jpg,male
3,data/1 (2191).jpg,male
4,data/1 (1746).jpg,male
